In [1]:
import os
import ants
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

2022-11-30 19:20:16.476520: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 19:20:16.624812: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-30 19:20:17.300562: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-30 19:20:17.300683: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [4]:
description = pd.read_csv('data/data.csv')
description.head()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I63874,941_S_1202,CN,M,78,1,MRI,MPR-R; GradWarp; B1 Correction; N3; Scaled,Processed,1/30/2007,NiFTI,NaN
1,I204843,941_S_1202,CN,M,81,8,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,3/14/2010,NiFTI,NaN
2,I137298,941_S_1202,CN,M,80,6,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,2/17/2009,NiFTI,NaN
3,I75150,941_S_1202,CN,M,78,3,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,8/24/2007,NiFTI,NaN
4,I105437,941_S_1202,CN,M,79,4,MRI,MPR; GradWarp; B1 Correction; N3; Scaled,Processed,2/28/2008,NiFTI,NaN


In [5]:
def preprocess_nii(filename, path):

    REG_Folder = 'data/registered_data/'
    Output_Folder = 'data/output/'

    fixed = "data/tpl-MNI305_T1w.nii.gz"
    head_mask = "data/tpl-MNI305_desc-head_mask.nii.gz"
    brain_mask = "data/tpl-MNI305_desc-brain_mask.nii.gz"
    moving = f"{path}/{filename}"

    image_ID = filename[:-4]
    row_index = description.index[description['Image Data ID'] == image_ID].tolist()[0]
    row = description.iloc[row_index]
    label = row['Group']

    labelled_reg_folder = os.path.join(REG_Folder, label, image_ID)
    labelled_output_file = os.path.join(Output_Folder, label, image_ID)

    ! antsRegistrationSyNQuick.sh -d 3 -f {fixed} -m {moving} -o {labelled_reg_folder}_ -p f -n 4
    ! antsBrainExtraction.sh -d 3 -a {labelled_reg_folder}_InverseWarped.nii.gz -e {head_mask} -m {brain_mask} -o {labelled_output_file}_{label}_
    for f in os.listdir(os.path.join(REG_Folder, label)):
        os.remove(os.path.join(REG_Folder, label, f))
    

In [7]:
db = 'data/raw_data'

for nii in os.listdir(db):
    try:
        preprocess_nii(nii, db)
    except RuntimeError:
        print('Exception with', os.path.join(nii))


--------------------------------------------------------------------------------------
 Mapping parameters
--------------------------------------------------------------------------------------
 ANTSPATH is /home/keef07/install/bin/

 Dimensionality:           3
 Output name prefix:       data/registered_data/CN/I98785_
 Fixed images:             data/tpl-MNI305_T1w.nii.gz
 Moving images:            data/raw_data/I98785.nii
 Mask images:              
 Initial transforms:       
 Number of threads:        4
 Spline distance:          26
 SyN gradient step:        0.1
 Transform type:           s
 MI histogram bins:        32
 Precision:                f
 Use histogram matching:   0
 Repro                     0
 antsRegistration call:
--------------------------------------------------------------------------------------
/home/keef07/install/bin//antsRegistration --verbose 1 --dimensionality 3 --float 1 --collapse-output-transforms 1 --output [ data/registered_data/CN/I98785_,data/regis

In [ ]:
test_split = 0.15
val_split = 0.15

class_subf = ['AD/', 'CN/', 'MCI/']
output_folder = "data/output/"

filenames = np.array([])

for subf in class_subf:
    path = output_folder + subf
    for name in os.listdir(path):
        complete_name = os.path.join(path, name)
        if os.path.isfile(complete_name):
            filenames = np.concatenate((filenames, complete_name), axis=None)

filenames.shape

In [ ]:
for i in range(1000):
  np.random.shuffle(filenames)
  
test_margin = int(len(filenames) * test_split)
training_set, test_set = filenames[test_margin:], filenames[:test_margin]

validation_margin = int(len(training_set) * val_split)
training_set, validation_set = training_set[validation_margin:], training_set[:validation_margin]

print('Training set:', training_set.shape)
print('Validation set:', validation_set.shape)
print('Test set:', test_set.shape)

In [ ]:
TFR_PATH = 'data/output/TFR'

TFR_TRAIN = 'train.tfrecords'
TFR_TEST = 'test.tfrecords'
TFR_VAL = 'val.tfrecords'

train_tfrec = os.path.join(TFR_PATH, TFR_TRAIN)
test_tfrec = os.path.join(TFR_PATH, TFR_TEST)
val_tfrec = os.path.join(TFR_PATH, TFR_VAL)

SHUFFLE_BUFFER = 10

LABELS = {'CN': 0, 'MCI': 1, 'AD': 2}

In [ ]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))
  
def _bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def load_image(abs_path):
    split_path = abs_path.split('/')
    label = LABELS[split_path[-2]]
    image_ID = split_path[-1].split("_")[0]
    ants_image = ants.image_read(abs_path)
    img = ants_image.numpy()
    
    return img, label, image_ID


def create_tf_record(img_filenames, tf_rec_filename):
    writer = tf.io.TFRecordWriter(tf_rec_filename)
    for meta_data in img_filenames:
        img, label, image_ID = load_image(meta_data)
        age_and_sex = description.loc[description['Image Data ID'] == image_ID, ['Age', 'Sex']].iloc[0]
        feature = {'label': _int64_feature(label),
               'subject_age': _int64_feature(age_and_sex[0]),
               'subject_sex': _bytes_feature(age_and_sex[1].encode('utf-8')),
               'image_id': _bytes_feature(image_ID.encode('utf-8')),
               'image': _float_feature(img.ravel())}
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
create_tf_record(training_set, train_tfrec)
create_tf_record(test_set, test_tfrec)

In [ ]:
from antspynet.architectures import create_resnet_model_3d
from tensorflow.keras import Sequential, layers, models
from tensorflow.keras import optimizers

def set_nontrainable_layers(model):
    model.trainable = False 
    return model

base_model = create_resnet_model_3d((172, 220, 156, 1), number_of_classification_labels=3)
base_model = set_nontrainable_layers(base_model)
flatten_layer = layers.Flatten()
dense_layer = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(3, activation='softmax')

model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])

model.summary()

In [ ]:
opt = optimizers.Adam(learning_rate=1e-3)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
def _decode(serialized_example):
    IMG_SHAPE = (172, 220, 156, 1)
    feature = tf.io.parse_single_example(
        serialized_example,
        features = {
            'image': tf.io.FixedLenFeature(IMG_SHAPE, tf.float32),
            'label': tf.io.FixedLenFeature([], tf.int64),
            'subject_sex': tf.io.FixedLenFeature([], tf.string),
            'subject_age': tf.io.FixedLenFeature([], tf.int64),
            'image_id': tf.io.FixedLenFeature([], tf.string)
        }
    )
    
    return feature['image'], feature['label']

In [ ]:
def dataset_parser(filepath, batch_size, n_classes=3):
    
    dataset = tf.data.TFRecordDataset(filepath).map(_decode)
    
    dataset = dataset.shuffle(SHUFFLE_BUFFER).repeat()
    dataset = dataset.batch(batch_size)  
    dataset = dataset.prefetch(1)
    
    iterator = dataset.__iter__()#make_one_shot_iterator()
    image, label = iterator.get_next()
    label = tf.one_hot(label, n_classes)
    
    return image, label

In [ ]:
TFR_PATH = 'data/output/TFR'

TFR_TRAIN = 'train.tfrecords'
TFR_TEST = 'test.tfrecords'
TFR_VAL = 'val.tfrecords'

train_tfrec = os.path.join(TFR_PATH, TFR_TRAIN)
test_tfrec = os.path.join(TFR_PATH, TFR_TEST)
val_tfrec = os.path.join(TFR_PATH, TFR_VAL)

SHUFFLE_BUFFER = 10

LABELS = {'CN': 0, 'MCI': 1, 'AD': 2}

In [ ]:
image_tensor, label_tensor = dataset_parser(train_tfrec, 5)

In [ ]:
val_image_tensor, val_label_tensor = dataset_parser(val_tfrec, 5)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=10, monitor="val_accuracy", mode="max", restore_best_weights=True)

history = model.fit(image_tensor, label_tensor, 
                    validation_data=(val_image_tensor, val_label_tensor), 
                    shuffle=True,
                    batch_size=16,
                    epochs=50,
                    callbacks=[es],
                    verbose=1)

In [ ]:
def plot_loss_accuracy(history):

    with plt.style.context('seaborn-deep'):

        fig, ax = plt.subplots(1, 2, figsize=(15, 4))

        ## Plot Losses and Accuracies
        x_axis = np.arange(len(history.history['loss']))

        ax[0].set_title("Loss")
        ax[0].plot(x_axis, history.history['loss'], color="blue", linestyle=":", marker="X", label="Train Loss")
        ax[0].plot(x_axis, history.history['val_loss'], color="orange", linestyle="-", marker="X", label="Val Loss")

        ax[1].set_title("Accuracy")
        ax[1].plot(x_axis, history.history['acc'], color="blue", linestyle=":", marker="X", label="Train Accuracy")
        ax[1].plot(x_axis,
                   history.history['val_acc'],
                   color="orange",
                   linestyle="-",
                   marker="X",
                   label="Val Accuracy")

        ## Customization
        ax[0].grid(axis="x", linewidth=0.5)
        ax[0].grid(axis="y", linewidth=0.5)
        ax[0].legend()
        ax[1].grid(axis="x", linewidth=0.5)
        ax[1].grid(axis="y", linewidth=0.5)
        ax[1].legend()

        plt.show()

In [ ]:
test_image_tensor, test_label_tensor = dataset_parser(test_tfrec, 5)
res = model.evaluate(test_image_tensor, test_label_tensor)[1]
res